<a href="https://colab.research.google.com/github/hhshanto/Dog-Breed-Identification/blob/main/Dog_Breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to end multiclass dog breed classification
This Notebook builds an end to end multiclass image classifier using TensorFlow 2.0 and TensorFlow Hub

## 1. Problem
Identifying the breed of a dog given and image of a dog.
## 2. Data
The data is available on Kaggle:
https://www.kaggle.com/c/dog-breed-identification/data
## 3. Evaluation
Submissions are evaluated on **Multi Class Log Loss** between the predicted probability and the observed target.
## 4. Features
Info about data:
* 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get our workspace ready

* Import TensorFlow Hub
* Make sure to use GPU

In [1]:
 # Import necessary tools
 import tensorflow as tf
 import tensorflow_hub as hub
 print("TF version:", hub.__version__)

 # Checking GPU availability
 print("GPU", "available (Yess!!)" if tf.config.list_physical_devices("GPU") else "not available")

TF version: 0.12.0
GPU available (Yess!!)
